In [2]:
import polars as pl

from pipelines.run_metrics import run_metrics

metrics = run_metrics(True,True)

In [8]:
metrics = pl.read_csv("preds/metrics.csv")
metrics

horizon,avaliability,features_type,rmse,target_name,model,rmse_naive,rel_rmse_naive
i64,i64,str,f64,str,str,f64,f64
1,1,"""d12""",0.013668,"""gdp_log_d4""","""gb""",0.021231,0.643763
1,1,"""rolling""",0.021978,"""gdp_log_d4""","""gb""",0.021231,1.035145
1,2,"""d12""",0.012233,"""gdp_log_d4""","""gb""",0.021231,0.576192
1,2,"""rolling""",0.015941,"""gdp_log_d4""","""gb""",0.021231,0.750826
1,3,"""d12""",0.009165,"""gdp_log_d4""","""gb""",0.021231,0.431678
…,…,…,…,…,…,…,…
1,3,"""d12""",0.072911,"""inv_cap_log_d4""","""dfm""",0.048473,1.504149
2,1,"""d12""",0.086723,"""inv_cap_log_d4""","""dfm""",0.043397,1.998359
5,1,"""d12""",0.073778,"""inv_cap_log_d4""","""dfm""",0.041417,1.781342


In [18]:
best_metrics = metrics.group_by(
    [
        'target_name',
        'horizon',
        'avaliability',
     ]
).agg(
    pl.col('rel_rmse_naive').min().alias('best_rel_rmse_naive'),
    pl.col('model').sort_by('rel_rmse_naive').first().alias('best_model'),
    pl.col('features_type').sort_by('rel_rmse_naive').first().alias('best_features_type'),
).sort(
    [
        'target_name',
        'horizon',
        'avaliability',
    ]
)
best_metrics

target_name,horizon,avaliability,best_rel_rmse_naive,best_model,best_features_type
str,i64,i64,f64,str,str
"""cons_log_d4""",1,1,0.529136,"""ngb""","""d12"""
"""cons_log_d4""",1,2,0.51962,"""gb""","""d12"""
"""cons_log_d4""",1,3,0.454168,"""gb""","""d12"""
"""cons_log_d4""",2,1,0.511075,"""ngb""","""d12"""
"""cons_log_d4""",2,2,0.721213,"""ngb""","""d12"""
…,…,…,…,…,…
"""inv_log_d4""",5,2,0.639336,"""gb""","""d12"""
"""inv_log_d4""",5,3,0.556782,"""dfm""","""d12"""
"""inv_log_d4""",6,1,0.607736,"""dfm""","""d12"""
